In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import numpy as np


First Model

In [4]:
# random rotation and horizontal flipping, epochs = 10, and learning rate scheduler to gradually reduce the learning rate during training.


from torch.optim.lr_scheduler import StepLR

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


transform = transforms.Compose([
    transforms.RandomRotation(15),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)
model = CNN()

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Learning rate scheduler
scheduler = StepLR(optimizer, step_size=1, gamma=0.1)

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 100 == 99:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0
    scheduler.step()

print('Finished Training')

# Evaluate the model
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        inputs, labels = data
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %.2f %%' % (100 * correct / total))

[1,   100] loss: 1.043
[1,   200] loss: 0.441
[1,   300] loss: 0.390
[1,   400] loss: 0.280
[1,   500] loss: 0.258
[1,   600] loss: 0.224
[1,   700] loss: 0.186
[1,   800] loss: 0.173
[1,   900] loss: 0.155
[2,   100] loss: 0.132
[2,   200] loss: 0.116
[2,   300] loss: 0.118
[2,   400] loss: 0.121
[2,   500] loss: 0.116
[2,   600] loss: 0.111
[2,   700] loss: 0.110
[2,   800] loss: 0.109
[2,   900] loss: 0.114
[3,   100] loss: 0.111
[3,   200] loss: 0.101
[3,   300] loss: 0.103
[3,   400] loss: 0.100
[3,   500] loss: 0.112
[3,   600] loss: 0.097
[3,   700] loss: 0.109
[3,   800] loss: 0.103
[3,   900] loss: 0.102
[4,   100] loss: 0.111
[4,   200] loss: 0.096
[4,   300] loss: 0.109
[4,   400] loss: 0.093
[4,   500] loss: 0.091
[4,   600] loss: 0.102
[4,   700] loss: 0.109
[4,   800] loss: 0.109
[4,   900] loss: 0.105
[5,   100] loss: 0.116
[5,   200] loss: 0.100
[5,   300] loss: 0.105
[5,   400] loss: 0.106
[5,   500] loss: 0.100
[5,   600] loss: 0.100
[5,   700] loss: 0.104
[5,   800] 

In [11]:
# Calculate total number of parameters
total_params = sum(p.numel() for p in model.parameters())

# Define input dimensions
input_dim = (1, 28, 28)

# Calculate compression ratio of the original dataset
t_size = np.prod(input_dim)
compression_ratio = total_params / t_size
print("The original dataset compression ratio is:", compression_ratio)

# Calculate compression ratio after the last layer
output_size = np.prod(model.fc2.out_features)  # Assuming model.fc2 is the last fully connected layer
compression_ratio_last_layer = output_size / t_size
print("The original dataset compression ratio after the last layer:", compression_ratio_last_layer)

# Calculate compression ratios for convolutional layers
compression_layer_ratios = []
for name, param in model.named_parameters():
    if 'conv' in name:
        input_size = np.prod(param.shape[1:])
        output_size = np.prod(param.shape[:2]) * np.prod(param.shape[3:])
        compression_ratio_layer = output_size / input_size
        compression_layer_ratios.append(compression_ratio_layer)

print("Compression ratios for convolutional layers:", compression_layer_ratios)


The original dataset compression ratio is: 537.8086734693877
The original dataset compression ratio after the last layer: 0.012755102040816327
Compression ratios for convolutional layers: [10.666666666666666, 32.0, 21.333333333333332, 64.0]


2nd Model

In [12]:
# Define custom CNN architecture
# CNN with two convolutional layers and two fully connected layers.
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Load the MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

# Initialize the model, loss function, and optimizer
model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 5
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 100 == 99:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

print('Finished Training')

# Evaluate the model
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        inputs, labels = data
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %.2f %%' % (100 * correct / total))

[1,   100] loss: 0.774
[1,   200] loss: 0.189
[1,   300] loss: 0.123
[1,   400] loss: 0.107
[1,   500] loss: 0.098
[1,   600] loss: 0.080
[1,   700] loss: 0.080
[1,   800] loss: 0.072
[1,   900] loss: 0.057
[2,   100] loss: 0.048
[2,   200] loss: 0.050
[2,   300] loss: 0.049
[2,   400] loss: 0.044
[2,   500] loss: 0.046
[2,   600] loss: 0.046
[2,   700] loss: 0.048
[2,   800] loss: 0.053
[2,   900] loss: 0.047
[3,   100] loss: 0.035
[3,   200] loss: 0.029
[3,   300] loss: 0.034
[3,   400] loss: 0.030
[3,   500] loss: 0.030
[3,   600] loss: 0.037
[3,   700] loss: 0.029
[3,   800] loss: 0.031
[3,   900] loss: 0.032
[4,   100] loss: 0.023
[4,   200] loss: 0.022
[4,   300] loss: 0.024
[4,   400] loss: 0.019
[4,   500] loss: 0.026
[4,   600] loss: 0.020
[4,   700] loss: 0.034
[4,   800] loss: 0.022
[4,   900] loss: 0.019
[5,   100] loss: 0.018
[5,   200] loss: 0.019
[5,   300] loss: 0.020
[5,   400] loss: 0.017
[5,   500] loss: 0.014
[5,   600] loss: 0.015
[5,   700] loss: 0.024
[5,   800] 

In [13]:
# Define input dimensions
input_dim = (1, 28, 28)

# Calculate compression ratio of the original dataset
t_size = np.prod(input_dim)
compression_ratio = total_params / t_size
print("The original dataset compression ratio is:", compression_ratio)

# Calculate compression ratio after the last layer
output_size = np.prod(model.fc2.out_features)  # Assuming model.fc2 is the last fully connected layer
compression_ratio_last_layer = output_size / t_size
print("The original dataset compression ratio after the last layer:", compression_ratio_last_layer)

# Calculate compression ratios for convolutional layers
compression_layer_ratios = []
for name, param in model.named_parameters():
    if 'conv' in name:
        input_size = np.prod(param.shape[1:])
        output_size = np.prod(param.shape[:2]) * np.prod(param.shape[3:])
        compression_ratio_layer = output_size / input_size
        compression_layer_ratios.append(compression_ratio_layer)

print("Compression ratios for convolutional layers:", compression_layer_ratios)

The original dataset compression ratio is: 537.8086734693877
The original dataset compression ratio after the last layer: 0.012755102040816327
Compression ratios for convolutional layers: [10.666666666666666, 32.0, 21.333333333333332, 64.0]
